# Google Drive File Integrity Checker

本工具可以检查 Google Drive 上传的文件的完整性。

步骤：

1. 本地使用 7-zip 输出 SHA256 文件
2. 使用 `check(hash_file)` 进行校验

In [1]:
import click
import os


def calculate_hash(file_path, buff_size=65536):
	import hashlib
	sha256 = hashlib.sha256()

	with open(file_path, 'rb') as f:
		with click.progressbar(label="Downloading from remote: ", length=f.tell()) as bar:
			while True:
				data = f.read(buff_size)
				if not data:
					break
				sha256.update(data)
				bar.update(len(data))

	return sha256.hexdigest()


def check(hash_file):
	path, file_name = os.path.split(hash_file)
	_, file_extension = os.path.splitext(file_name)

	if file_extension != '.sha256':
		raise ValueError('File is not a sha256 file')

	with open(hash_file, 'r') as f:
		hash_info = f.read()

	hash_file_calculated = {}
	for hash_info_line in hash_info.splitlines():
		hash_info = hash_info_line[:64]
		hash_file_calculated[os.path.join(path, hash_info_line[66:])] = hash_info

	mismatched_files = []

	for file_path, file_hash in hash_file_calculated.items():
		if not os.path.isfile(file_path):
			raise ValueError('File {} does not exist'.format(file_path))
		print(f'Calculating hash for {file_path} ...')
		res_hash = calculate_hash(file_path)
		print(f'Calculated: {res_hash}')
		print(f'Expected  : {file_hash}')
		if res_hash != file_hash:
			print(f'ERROR: HASH DOES NOT MATCH for {file_path}')
			mismatched_files.append(file_path)

	if len(mismatched_files) > 0:
		print(f'{len(mismatched_files)} files mismatched')
		for file_path in mismatched_files:
			print(f'  {file_path}')
	else:
		print('All files matched')
